<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>


# GitHub - Add new Github issues as page in Notion database


**Tags:** #github #notion #operations #automation


**Author:** [Sanjeet Attili](https://linkedin.com/in/sanjeet-attili-760bab190/)

This notebook is used to create Notion page in a database when a new Github issue is created.

<br/>References :
- Notion driver:
- Github SDK: [https://github.com/PyGithub/PyGithub](https://github.com/PyGithub/PyGithub)


## Input


### Import libraries


In [1]:
import naas
from naas_drivers import notion, github

### Setup Notion


<a href='https://docs.naas.ai/drivers/notion'>How to get your Notion integration token ?</a>

In [8]:
# Notion token
notion_token = "..."

# Specify database id
notion_database_id = "..."

notion_client = notion.connect(notion_token)


### Setup Github


In [9]:
# Github token
github_token = "..."

# Github repo on which we want to create issues.
github_repo_name = "..."

gh_client = github.connect(github_token)


### Setup Naas


In [ ]:
# Schedule your notebook every 15 minutes.
naas.scheduler.add(cron="*/15 * * * *")

#-> Uncomment the line below to remove your scheduler
# naas.scheduler.delete()


## Model


### Query notion database


In [14]:
db_notion = notion_client.database.get(notion_database_id)
df_notion = db_notion.df()

for idx, row in df_notion.iterrows():
    if row.Name=='':
        df_notion.drop(index= idx, inplace=True)
    try:
        df_notion.drop(columns=['Tags'], inplace=True)
    except KeyError:
        pass

df_notion

,URL,Last_created,Assignees,Issue_id,Name
0,https://github.com/jupyter-naas/awesome-notebo...,2021-05-26T15:15:00.000+00:00,None,902545370,LinkedIn - Get company followers
1,https://github.com/jupyter-naas/awesome-notebo...,2021-05-30T11:39:00.000+00:00,FlorentLvr,906727513,Pillow - Add data to an image
2,https://github.com/jupyter-naas/awesome-notebo...,2021-06-02T21:46:00.000+00:00,None,909910813,Odoo - Connect to API
3,https://github.com/jupyter-naas/awesome-notebo...,2021-06-02T22:39:00.000+00:00,iair,909939228,Fitbit - Connect to API
4,https://github.com/jupyter-naas/awesome-notebo...,2021-06-02T22:40:00.000+00:00,None,909939700,Oracle DB - Connect to API
...,...,...,...,...,...
56,https://github.com/jupyter-naas/awesome-notebo...,2022-04-10T05:40:00.000+00:00,SanjuEpic,1198888486,421 slack follow number of slack users in work...
57,https://github.com/jupyter-naas/awesome-notebo...,2022-04-10T06:24:00.000+00:00,SanjuEpic,1198901827,feat: github issues driver added
58,https://github.com/jupyter-naas/awesome-notebo...,2022-04-10T18:18:00.000+00:00,SanjuEpic,1199131652,427 linkedin follow total content views
59,https://github.com/jupyter-naas/awesome-notebo...,2022-04-11T12:21:00.000+00:00,SanjuEpic,1199892649,426 linkedin follow number of content published


### Get Github issues


In [11]:
df_issues = gh_client.repos.get_issues(github_repo_name)
df_issues.head()

,link_to_the_issue,issue_number,issue_title,issue_state,issue_id,issue_labels,issue_assignees,comments_till_date,last_created_date,last_created_time,last_updated_date,last_updated_time,comments,linked_pr_state,PR_activity
0,https://github.com/jupyter-naas/awesome-notebo...,450,Twitter - Post analysis,open,1200193149,None,calok64,0,2022-04-11,16:04:37,2022-04-11,16:04:52,No comments,None,None
1,https://github.com/jupyter-naas/awesome-notebo...,449,426 linkedin follow number of content published,open,1199892649,None,SanjuEpic,0,2022-04-11,12:21:47,2022-04-11,12:21:48,No comments,open,No activity since 0 days
2,https://github.com/jupyter-naas/awesome-notebo...,447,427 linkedin follow total content views,open,1199131652,None,SanjuEpic,0,2022-04-10,18:18:25,2022-04-10,18:18:25,No comments,open,No activity since 1 days
3,https://github.com/jupyter-naas/awesome-notebo...,446,feat: github issues driver added,open,1198901827,None,SanjuEpic,2,2022-04-10,06:24:51,2022-04-11,16:38:36,"['Hello @SanjuEpic, are you sure your code is ...",open,No activity since 0 days
4,https://github.com/jupyter-naas/awesome-notebo...,445,421 slack follow number of slack users in work...,open,1198888486,None,SanjuEpic,2,2022-04-10,05:40:47,2022-04-11,16:46:05,"['Hello @SanjuEpic, I tested your notebook and...",open,No activity since 0 days


### Create notion page from Github issue


In [12]:
def notion_page_from_gh_issue(issue, notion_database_id):
    
    if len(df_notion) == 0:
        pass
    elif len(df_notion) != 0 and str(issue.issue_id) in df_notion.Issue_id.to_list():
        return "issue already exists in database"
    
    page = notion_client.page.create(database_id=notion_database_id, title = issue.issue_title)
    page.rich_text("URL",issue.link_to_the_issue)
    page.rich_text("Assignees",issue.issue_assignees)
    page.number('Issue_id',issue.issue_id)
    page.date('Last_created',str(issue.last_created_date)+'T'+str(issue.last_created_time))
    page.update()
    
    return "Done"


## Output


### Iterate over each issue.


In [15]:
for idx, issue in df_issues.iterrows():
    val = notion_page_from_gh_issue(issue, notion_database_id)
    if val == "issue already exists in database":
        print("Database up to date!")
        break
    print(f'✅ Notion page created for issue {issue.link_to_the_issue}')


Database up to date!
